## Bird Species Identification

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'G:\DATASET\Images'
valid_path = 'G:\DATASET\Images'

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
 # useful for getting number of classes
folders = glob('G:\DATASET\Images\*')

  

# our layers
x = Flatten()(vgg.output)

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('G:\DATASET\Images',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('G:\DATASET\Images',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


print(len(training_set))
print(len(test_set))
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),

)


import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')w


Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [23]:
test_generator = train_datagen.flow_from_dataframe(test,directory='G:\DATASET\Test',x_col = 'Image',
                 y_col = None,target_size=(224,224),class_mode = None,batch_size=32)

Found 20 validated image filenames.


In [19]:
test_x=model.predict(test_generator) 
test_x=np.array(test_x)

In [27]:
test_x

array([[1.42399194e-05, 1.61943876e-06, 7.79175878e-01, 6.19171351e-06,
        1.07124761e-05, 1.54219947e-06, 6.68425419e-05, 1.55044962e-02,
        2.05218449e-01],
       [1.76869332e-06, 2.04231264e-06, 3.37020168e-03, 2.07248522e-05,
        6.67900968e-05, 2.11667102e-05, 7.23700214e-05, 3.58983903e-04,
        9.96086001e-01],
       [6.87570637e-03, 4.09215834e-04, 1.08858414e-01, 3.41501758e-02,
        7.16322362e-02, 8.06160830e-03, 1.54545745e-02, 5.85629046e-03,
        7.48701751e-01],
       [1.14405602e-05, 1.16908086e-04, 6.92311078e-02, 1.05238702e-04,
        7.22834666e-05, 7.83508076e-05, 6.17498416e-04, 2.14746315e-03,
        9.27619755e-01],
       [1.24505605e-03, 9.14837525e-04, 3.82199176e-02, 9.28205326e-02,
        1.05475197e-02, 6.90984773e-03, 1.43842092e-02, 7.66752958e-02,
        7.58282840e-01],
       [6.18177513e-03, 7.45000243e-01, 3.44593148e-03, 2.80807074e-02,
        7.80771673e-02, 6.13715313e-02, 1.31835919e-02, 6.24148324e-02,
        2.2

In [20]:
out=np.argmax(test_x,axis=1)

In [21]:
out

array([2, 8, 8, 8, 8, 1, 8, 3, 8, 8, 8, 8, 0, 8, 8, 2, 2, 7, 0, 8],
      dtype=int64)

In [26]:
import pandas as pd
import os
test=pd.DataFrame()
test_image=os.listdir('G:\DATASET\Test')
test['Image']=test_image
test['class']=out
test.head(20)

,Image,class
0,TEST.jpg,2
1,test06.jpg,8
2,test1.jpg,8
3,test10.jpg,8
4,test11.jpg,8
5,test12.jpeg,1
6,test13.jpg,8
7,test14.jpg,3
8,test15-16.jpg,8
9,test15.jpg,8


In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')